In [ ]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from scipy import stats
from nltk.metrics.scores import f_measure
import numpy as np
import torchmetrics as torchm
import nltk 
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
wer = torchm.WordErrorRate()
cer = torchm.CharErrorRate()
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

def sets(refs, preds):
    refs_set = set()
    preds_set = set()
    r_words = tokenizer.tokenize(refs.lower())
    for w_r in r_words:
        if w_r not in refs_set:
            refs_set.add(w_r)
    p_words = tokenizer.tokenize(preds.lower())
    for w_p in p_words:
        if w_p not in preds_set:
            preds_set.add(w_p)
    

    return refs_set, preds_set

In [ ]:
df = pd.read_csv(input('Introduce the translation training data: '), sep = '\t')
df = df[['source','translation','evaluator1 scores', 'evaluator2 scores']]


In [ ]:
wer_list, cer_list, f_list = [], [], []
n = 0
for src, tgt in zip(df['source'], df['translation']):
#     print(n)
    a = time.time()
#     if n == 100:
#         break
    print(len(src),len(tgt))
    wer_score = wer(target=src, preds=tgt).item() * 100
    wer_list.append(wer_score)
    cer_score = cer(target=src, preds=tgt).item() * 100
    cer_list.append(cer_score)
    r_s, p_s = sets(src, tgt)
    f_score = f_measure(r_s, p_s) * 100
    f_list.append(f_score)
    n+=1
    print(time.time()-a)

In [ ]:
df['wer'] = wer_list
df['cer'] = cer_list
df['f_score'] = f_list

In [ ]:
x = df[['wer','cer','f_score']]
y = df['evaluator scores']
regr = linear_model.LinearRegression()

x_ = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x)
regr.fit(x_, y)

predictions = regr.predict(input('Introduce the translation test data: ')) 

print('Pearson corr: ',stats.pearsonr(df['evaluator scores'],predictions)[0])